<a href="https://colab.research.google.com/github/brianhphillips/testrepo/blob/main/IMDb_Screenwriter_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Set base path
base_path = '/content/drive/MyDrive/'  # update this path accordingly

# Load the relevant datasets
name_basics = pd.read_csv(base_path + 'name.basics.tsv', sep='\t', na_values='\\N', dtype=str)
title_basics = pd.read_csv(base_path + 'title.basics.tsv', sep='\t', na_values='\\N', dtype=str)
title_crew = pd.read_csv(base_path + 'title.crew.tsv', sep='\t', na_values='\\N', dtype=str)
title_ratings = pd.read_csv(base_path + 'title.ratings.tsv', sep='\t', na_values='\\N', dtype=str)

Mounted at /content/drive


In [2]:
# List of screenwriters
screenwriters = [
    "James Cameron", "Steve Kloves", "Christopher Markus", "Stephen McFeely",
    "Peter Jackson", "Philippa Boyens", "Andrew Stanton", "Chris McKenna",
    "Amanda Silver", "Neal Purvis", "Erik Sommers", "Rick Jaffa", "J.J. Abrams",
    "George Lucas", "Michael Arndt", "Jared Bush", "Colin Trevorrow",
    "Meg LeFauve", "Jonathan Nolan", "Matthew Fogel", "Drew Pearce",
    "Daniel Gerson", "Eric Darnell", "J.K. Rowling"
]

# Filter name.basics to get nconsts for the screenwriters
screenwriter_df = name_basics[name_basics['primaryName'].isin(screenwriters)][['nconst', 'primaryName']]

In [3]:
# Explode writers list in title.crew
title_crew_filtered = title_crew.dropna(subset=['writers']).copy()
title_crew_filtered['writers'] = title_crew_filtered['writers'].str.split(',')

# Explode to one row per writer
title_crew_exploded = title_crew_filtered.explode('writers')

# Filter to only screenwriters' nconsts
written_titles = title_crew_exploded[title_crew_exploded['writers'].isin(screenwriter_df['nconst'])]

In [4]:
# Merge ratings with the written titles
written_titles_ratings = written_titles.merge(title_ratings, on='tconst')

# Now join with screenwriter names for readability
written_titles_ratings = written_titles_ratings.merge(
    screenwriter_df,
    left_on='writers',
    right_on='nconst'
)

# Convert rating to float
written_titles_ratings['averageRating'] = written_titles_ratings['averageRating'].astype(float)

# Group by writer and compute average
avg_ratings = written_titles_ratings.groupby('primaryName')['averageRating'].mean().reset_index()
avg_ratings = avg_ratings.sort_values(by='averageRating', ascending=False)

In [5]:
print(avg_ratings)

           primaryName  averageRating
14      Jonathan Nolan       8.662667
13          Jared Bush       8.239726
20     Philippa Boyens       8.000000
10         J.J. Abrams       7.958114
17       Michael Arndt       7.627273
19       Peter Jackson       7.603846
2        Chris McKenna       7.590000
3   Christopher Markus       7.574286
22     Stephen McFeely       7.574286
7         Eric Darnell       7.555615
23        Steve Kloves       7.356250
5        Daniel Gerson       7.206667
9         George Lucas       7.179239
11        J.K. Rowling       7.157447
1       Andrew Stanton       7.071429
8         Erik Sommers       7.044444
12       James Cameron       6.926994
6          Drew Pearce       6.800000
18         Neal Purvis       6.785000
16         Meg LeFauve       6.716667
4      Colin Trevorrow       6.642857
0        Amanda Silver       6.636842
21          Rick Jaffa       6.600000
15       Matthew Fogel       6.125000
